In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchtext
import nltk
from konlpy.tag import Kkma
from torchtext.data import Field, BucketIterator, TabularDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

## 데이터 준비 

대화 모델 더미 데이터

In [2]:
kor_tagger = Kkma()

kor_tagger = kor_tagger.morphs

In [3]:
SOURCE = Field(tokenize=kor_tagger,use_vocab=True,init_token="<s>",eos_token="</s>",lower=True, include_lengths=True, batch_first=True)
TARGET = Field(tokenize=kor_tagger,use_vocab=True,init_token="<s>",eos_token="</s>",lower=True, batch_first=True)

In [4]:
train_data = TabularDataset(
                                   path="data/dsksd_chat.txt",
                                   format='tsv', # \t로 구분
                                   #skip_header=True, # 헤더가 있다면 스킵
                                   fields=[('inputs',SOURCE),('targets',TARGET)])

In [5]:
SOURCE.build_vocab(train_data)
TARGET.build_vocab(train_data)

In [6]:
print(len(SOURCE.vocab),len(TARGET.vocab))

316 272


In [7]:
train_loader = BucketIterator(
    train_data, batch_size=32, device=-1, # device -1 : cpu, device 0 : 남는 gpu
    sort_key=lambda x: len(x.inputs),sort_within_batch=True,repeat=False) 

In [8]:
for batch in train_loader:
    break

In [13]:
inputs, lengths = batch.inputs

## PackedSequence 

In [11]:
E = 50
H = 100

In [12]:
embed = nn.Embedding(len(SOURCE.vocab),E)
rnn = nn.GRU(E,H,batch_first=True)

In [14]:
embedded = embed(inputs)
print(embedded.size())

torch.Size([32, 8, 50])


In [15]:
rnn(embedded)

(Variable containing:
 ( 0 ,.,.) = 
  -1.3902e-01 -2.7971e-01  2.8283e-02  ...   2.9061e-02  2.0862e-01 -2.1951e-01
  -1.0113e-01 -3.4135e-01  9.2160e-02  ...   1.1204e-01  2.1351e-01 -9.4129e-02
  -1.9690e-01 -1.6823e-01  4.1245e-01  ...   3.2552e-03  2.7113e-02 -6.2665e-02
                  ...                   ⋱                   ...                
  -2.7761e-01 -2.3033e-01 -3.0036e-01  ...   1.3460e-01 -3.8957e-02  9.4862e-02
  -1.3562e-01  1.7226e-04 -9.1728e-02  ...   1.3502e-01 -1.5232e-01  1.0153e-01
  -1.0844e-01  3.6799e-02 -1.3009e-01  ...   2.4265e-02 -4.7897e-01  6.5472e-02
 
 ( 1 ,.,.) = 
  -1.3902e-01 -2.7971e-01  2.8283e-02  ...   2.9061e-02  2.0862e-01 -2.1951e-01
  -7.8600e-02 -4.6822e-01 -2.4966e-01  ...   4.3685e-02 -4.6790e-02 -8.7919e-03
   1.9377e-01 -1.8793e-01  4.6660e-02  ...   2.0945e-01  3.2350e-01  2.5769e-01
                  ...                   ⋱                   ...                
   1.6942e-01 -2.3333e-01 -5.0149e-02  ...  -1.1582e-01 -6.0621e-03 

## Encoder 

In [96]:
class Encoder(nn.Module):
    def __init__(self,V,E,H,num_layers=1,bidirec=False):
        super(Encoder,self).__init__()
        
        self.hidden_size = H
        self.num_layers = num_layers
        self.num_directions = 2 if bidirec else 1
        
        self.embed = nn.Embedding(V,E)
        self.dropout = nn.Dropout(0.5)
        self.gru = nn.GRU(E,H,num_layers,batch_first=True,bidirectional=bidirec)
        
    def init_hidden(self,batch_size):
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        return hidden
    
    def forward(self,inputs,input_lengths):
        """
        inputs : B,T # LongTensor
        input_lengths : B # list
        """
        
        embed = self.embed(inputs)
        embed = self.dropout(embed)
        hidden = self.init_hidden(inputs.size(0))
        
        # 패딩된 문장을 패킹(패딩은 연산 안들어가도록)
        packed = pack_padded_sequence(embed, input_lengths,batch_first=True) 
        output, hidden = self.gru(packed,hidden)
        
        # 패킹된 문장을 다시 unpack
        output, output_lengths = pad_packed_sequence(output,batch_first=True) 
        
        # last hidden 선택하기 , concat
        hidden = hidden[-self.num_directions:] # num_layers*num_directions,batch_size,hidden_size
        hidden = torch.cat([h for h in hidden],1).unsqueeze(0) # 1,B,2H
        
        return output, hidden

## Decoder 

In [97]:
class Decoder(nn.Module):
    def __init__(self,V,E,H,sos_idx,max_len=15):
        super(Decoder,self).__init__()
        
        self.hidden_size = H
        self.max_len = max_len
        self.sos_idx = sos_idx
        self.embed = nn.Embedding(V,E)
        self.gru = nn.GRU(E,H,batch_first=True)
        self.linear = nn.Linear(H,V)
        
    def start_token(self,batch_size):
        return Variable(torch.LongTensor([self.sos_idx]*batch_size)).unsqueeze(1)
        
    def forward(self,hidden, max_len=None):
        
        if max_len is None: max_len = self.max_len
        
        inputs = self.start_token(hidden.size(1)) # Batch_size
        embed = self.embed(inputs)
                    
        scores=[]
        for _ in range(max_len):
            _, hidden = self.gru(embed,hidden)
            score = self.linear(hidden.squeeze(0))
            scores.append(score)
            decoded = score.max(1)[1]
            embed = self.embed(decoded).unsqueeze(1) # y_{t-1}
            
        #  column-wise concat, reshape!!
        scores = torch.cat(scores,1)
        return scores.view(inputs.size(0)*max_len,-1)

## Train 

In [101]:
HIDDEN = 30
EMBED = 30
STEP = 150
LR = 0.01

encoder = Encoder(len(SOURCE.vocab),EMBED,HIDDEN,bidirec=True)
decoder = Decoder(len(TARGET.vocab),EMBED,HIDDEN*2,TARGET.vocab.stoi['<s>'])

loss_function = nn.CrossEntropyLoss(ignore_index=TARGET.vocab.stoi['<pad>'])
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()),lr=LR)

In [102]:
encoder.train()
for step in range(STEP):
    losses=[]
    for i, batch in enumerate(train_loader):
        inputs,lengths = batch.inputs
        targets = batch.targets
        
        encoder.zero_grad()
        decoder.zero_grad()
        
        output, hidden = encoder(inputs,lengths.tolist())
        preds = decoder(hidden,targets.size(1))
        
        loss = loss_function(preds,targets.view(-1))
        losses.append(loss.data[0])
        
        loss.backward()
        optimizer.step()
        
    if step % 10 == 0:
        print(np.mean(losses))
        losses=[]

5.215575122833252
3.305179214477539
2.783242416381836
2.3035650730133055
2.0305253982543947
1.6812453031539918
1.4967052459716796
1.191986608505249
0.9598398089408875
0.8565816164016724
0.6940780758857727
0.6236866474151611
0.5090916633605957
0.45437546968460085
0.43694338798522947


## TEST 

In [103]:
encoder.eval()
while 1:
    try:
        text = input()
        tokenized = ["<s>"] + kor_tagger(text) +["</s>"]
        input_,length = SOURCE.numericalize(([tokenized],[len(tokenized)]),train=False,device=-1)

        o,h = encoder(input_,length.tolist())
        preds = decoder(h)
        reply = [TARGET.vocab.itos[i] for i in preds.max(1)[1].data.tolist() if i not in [0,1,2,3]]

        print(" ".join(reply))
    except KeyboardInterrupt as e:
        break

파이토치 좋아?
한번 사용 하 어 보시 면 알 시 ㄹ 거 에 요 !
뭐해?
저 어 어 어 어 이 요 ?
ㅡㅡ
알 겠 !!
꺼져
요 어 어 이 요 ?
바보네
말 이 거치 시 네요 .. ㅠ
잘가
아 알 !! 요 요
ㅗ
요
ㅋㅋ
감사 !
